In [1]:
import numpy as np
import scipy.sparse as sps
import porepy as pp

In [2]:
import sys
sys.path.append("./class")
from Transport_class import Transport

In [3]:
Nx=4
#Ny=2
phys_dims=0.3
#phys_dims=[1.0,2.0]
#g=pp.CartGrid([Nx,Ny],phys_dims)
g=pp.CartGrid([Nx],phys_dims)
g.compute_geometry()            

In [4]:
unity = np.ones(g.num_cells)
empty = np.empty(0)
porosity=unity
aperture=1
bc_type=["dir","dir"]
bc_value=[0.,0.]
init_cond=lambda x,y,z:int(x<0.5)
print(g.num_faces)

5


In [5]:
specified_parameters = {
            "bc_type": bc_type,
            "bc_value": bc_value,
            "time_step": 0.01,
            "mass_weight": porosity * aperture,
            #"darcy_flux":np.zeros(g.num_faces),
            "darcy_flux":np.ones(g.num_faces),
            "t_max": 1,
            "method": "Explicit",
            "lambda_lin_decay":0,
            "initial_cond":init_cond,
}

In [6]:
#specified_parameters["darcy_flux"][:(Nx+1)*Ny]=1.

In [7]:
#specified_parameters["darcy_flux"][(Nx+1)*Ny:]=2.

In [8]:
vel=specified_parameters["darcy_flux"]
print("vel")
print(vel)

vel
[1. 1. 1. 1. 1.]


In [9]:
transport=Transport(g,specified_parameters)

In [10]:
transport.set_bc()

In [11]:
tracer=np.zeros(g.num_cells)

In [12]:
transport.set_initial_cond(tracer)

In [13]:
data=transport.data[pp.PARAMETERS]["transport"]
kw_t="transport"
node_discretization = pp.Upwind(kw_t)
source_discretization = pp.ScalarSource(kw_t)
mass_discretization = pp.MassMatrix(kw_t)
        
node_discretization.discretize(transport.g,transport.data)
source_discretization.discretize(transport.g,transport.data)
mass_discretization.discretize(transport.g,transport.data)
        
A_upwind,b_upwind=node_discretization.assemble_matrix_rhs(transport.g,transport.data)
        #_,b_source=source_discretization.assemble_matrix_rhs(self.g,self.data)
A_mass,b_mass=mass_discretization.assemble_matrix_rhs(transport.g,transport.data)


In [14]:
print("A_mass:")
print(A_mass)


A_mass:
  (0, 0)	0.075
  (1, 1)	0.075
  (2, 2)	0.07499999999999998
  (3, 3)	0.07500000000000001


In [15]:
print("A_upwind:")
print(A_upwind)

A_upwind:
  (0, 0)	1.0
  (1, 1)	1.0
  (1, 0)	-1.0
  (2, 2)	1.0
  (2, 1)	-1.0
  (3, 3)	1.0
  (3, 2)	-1.0


In [16]:
print("b_upwind")
print(b_upwind)

b_upwind
[0. 0. 0. 0.]
